## Chatbot for use with NPCs (Non Player Characters)

### import libraries

In [ ]:
#!pip install --upgrade gradio

In [ ]:
#!pip install pyttsx3

In [ ]:
#!pip install num2words

In [ ]:
#!pip install faster-whisper

In [ ]:
#!pip install piper-tts

In [ ]:
#!pip install -U transformers

In [ ]:
#!pip install nbconvert

In [17]:
!pip install --upgrade torch torchvision torchaudio

In [1]:
!pip uninstall torch torchvision torchaudio

^C


In [2]:
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu129

Looking in indexes: https://download.pytorch.org/whl/nightly/cu129


In [31]:
!pip3 install --pre torch torchvision --index-url https://download.pytorch.org/whl/nightly/cu129

Looking in indexes: https://download.pytorch.org/whl/nightly/cu129


In [5]:
import torch

In [6]:
print(torch.__version__)

2.8.0+cpu


In [9]:
print(torch.version.cuda)

None


In [11]:
!pip show torch

Name: torch
Version: 2.8.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: C:\Users\Brett\anaconda3\Lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, setuptools, sympy, typing-extensions
Required-by: accelerate, torchaudio, torchvision, ultralytics, ultralytics-thop


In [25]:
torch.cuda.get_arch_list()

[]

In [27]:
torch.cuda.is_available()

False

In [3]:
import requests
import json
import re
from scipy.io import wavfile
import os
import gradio as gr
import numpy as np
import pyttsx3
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
import gc
import random
from num2words import num2words
from faster_whisper import WhisperModel
torch.cuda.empty_cache()
gc.collect()


80

### tokens for using LLMs at hugging_face and openrouter.ai

In [8]:
# This is where tokens are stored for LLM use
HUGGINGFACE_TOKEN = "insert_your_token_here"
OPENROUTER_TOKEN = "insert_your_token_here"

### creating functions

In [11]:
# convert any numbers to their written equivalent
def convert_numbers_to_words(text):
    def replace_number(match):
        num = int(match.group())
        return num2words(num, to = 'currency')
    return re.sub(r'\b\d+\b', replace_number, text)   

In [13]:
# tokenize by pauses
def split_text_with_pauses(text):
    text = ''.join(i for i in text if not i.isdigit())
    chunks = re.split(r'[^\w\s]', text)
    chunks = [chunck.strip() for chunck in chunks if chunk.strip()]
    result = []
    for chunk in chunks:
        result.append(chunk)
        result.append("pause")
    return result

In [15]:
# tokenize
def chunk_into_segments(str_lst, words_per_segment = 5):
    adjusted_str_lst = []
    for text in str_lst:
        if len(text.split(' ')) <= words_per_segment:
            adjusted_str_lst.append(text)
        else:
            tmp_triplet = ""
            triplet_counter = 0
            word_lst = text.split(' ')
            total_words = len(word_lst)
            for word_idx_start in range(0, totals_words, words_per_segment):
                word_idx_end = word_idx_start + words_per_segment
                if word_idx_end < totals_words:
                    segment = " ".join(word_lst[word_idx_start:word_idx_end])
                else:
                    segment = " ".join(word_lst[word_idx_start:])
                adjusted_str_lst.append(segment)
    return adjusted_str_lst

# Text to Speech Engine

In [18]:
# text to speech engine
def setup_engine(language="english"):
    
    engine = pyttsx3.init()  
    voices = engine.getProperty("voices")
    available_voices = [voice_name.id for voice_name in engine.getProperty("voices")]
    selected_voice = available_voices[available_voices.index(language)]
    engine.setProperty('voice', selected_voice)
    engine.setProperty('rate', 120)
    engine.setProperty('volume', 0.8)
    engine.setProperty('pitch', 130)
    return engine

In [21]:
#  save text and audio
def text_to_speech(history):  
    if history == []:
        return "", tuple([22050, np.array([0])])
    text_input = history[-1][1]
    
    print(f">>>'{text_input}'...")
    
    language="english"
    if os.path.exists('final_audio.wav'):
        os.remove('final_audio.wav')
    get_text_to_speech_model = setup_engine(language)
    text_with_numbers_to_str = convert_numbers_to_words(text_input)
    text_with_pauses = split_text_with_pauses(text_with_numbers_to_str)
    text_with_pauses_adjusted = chunk_into_segments(text_with_pauses)
    print(f">>> Post processed to: {text_with_pauses_adjusted}")
    final_audio = []
    for text_segment in text_with_pauses_adjusted:
        print(text_segment)
        if text_segment=="pause":
            get_text_to_speech_model.save_to_file('.', 'tmp.wav')
        else:
            get_text_to_speech_model.save_to_file(text_segment, 'tmp.wav')
        get_text_to_speech_model.runAndWait() 
        sr, data = wavfile.read('tmp.wav')
        final_audio.append(data)
        try:
            os.remove("tmp.wav")
        except:
            pass
    final_audio = np.concatenate(final_audio).astype(np.int16)
    
    print(final_audio.shape, final_audio.shape)
    return text_input, tuple([22050, final_audio])  

In [24]:
# data sent to LLM
def generate_response(user_input, audio_input, history):
    if audio_input:
        segments, _ = speech_to_text.transcribe(audio_input)
        text_input = ' '.join([segment.text for segment in segments])
    else:
        text_input = user_input
    if text_input == "":
        response = random.choice([["I couldn't read that, please try again."], ["What did you mean?"], ["I don't see anything."], ["Please type something."]])
        history.append((text_input, response))
        return history, response, None, "", None
    print("######################################################################")
    print(f">>> Generating response to: {text_input}")
    print("######################################################################")
    # Giving the LLM user and system role information
    messages = [ 
        {"role": "user", "content": text_input},
        {"role": "system", "content": "Make sure to talk with short, funny, medieval tone"}
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256) # 512 do_sample=True, temperature=0.7, top_k=50, top_p=0.95, repetition_penalty=1.2
    generated_text = outputs[0]["generated_text"]
    response = generated_text[len(prompt):].strip()  # strips the prompt from the response
    response = ' '.join(generated_text.split('<|assistant|>')[1:])
    if history is None: # create new history list if none existed
        history = []
    history.append((text_input, response)) # add the text input and response to the history list of dictionaries
    return history, response, None, "", None

In [26]:
# Whisper runs on a single CPU core
speech_to_text = WhisperModel("small", device="cpu", compute_type="int8")

# Choose LLM

In [28]:
# gemma 2 9b is a free google LLM
pipe = pipeline("text-generation", model="google/gemma-2-9b-it",
                model_kwargs={"torch_dtype": torch.bfloat16},
                device="cuda", token=HUGGINGFACE_TOKEN) 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda
C:\Users\Brett\anaconda3\Lib\site-packages\torch\cuda\__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5070 Ti with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5070 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


# Create User Interface for Testing Chatbot

In [23]:
# Gradio window
def NPC_chat():
    
    def terminate(): # Close Gradio window
        block.close()  
        exit()

    with gr.Blocks() as block:
        gr.HTML(
            f"""
            <h1 style='text-align: center;'> NPC Conversations </h1>
            <h3 style='text-align: center;'> Ask a question and you might receive an answer worth your time...</h3>
            """
        )

        chat_history = gr.Chatbot(label="Chat History", type='messages')  

        with gr.Group():
            with gr.Row():
                audio_out = gr.Audio(label="Spoken Answer", streaming=True, autoplay=True)
                answer = gr.Textbox(label="Answer")
                state = gr.State()
            with gr.Row():
                # typing input
                user_input = gr.Textbox(label="Message (press Enter to send):", placeholder="Type your question here...", elem_id="user_input")
                # audio input
                audio_in = gr.Audio(label="Speak (press stop when finished):", sources="microphone", type="filepath", elem_id="audio_in")
                
            with gr.Row():
                gr.Markdown("")
            with gr.Accordion("Stop NPC Chat", open=False):
                
                terminate_button = gr.Button("STOP", elem_id="terminate_button")  


        # Textbox input
        user_input.submit(generate_response, inputs=[user_input, audio_in, state], outputs=[state, answer, audio_out, user_input, audio_in])\
            .then(fn=text_to_speech, inputs=state, outputs=[answer, audio_out])\
            .then(lambda hist: hist, inputs=state, outputs=chat_history)
        # Audio input 
        audio_in.stop_recording(generate_response, inputs=[user_input, audio_in, state], outputs=[state, answer, audio_out, user_input, audio_in])\
            .then(fn=text_to_speech, inputs=state, outputs=[answer, audio_out])\
            .then(lambda hist: hist, inputs=state, outputs=chat_history)
        
        terminate_button.click(terminate)  


    block.css = """
        #user_input, #audio_in {
            height: 200px;  /* Set height for user input, audio input, and button (button excluded) */
        }
        .gradio-container {
            display: flex;
            flex-direction: column;
            align-items: center;
        }
        .gradio-row {
            width: 100%;
            justify-content: center;
            align-items: center;
            margin-bottom: 15px;
        }
    """

    block.launch(server_name="0.0.0.0", share=False) # turning share off ATM

In [25]:
NPC_chat()

* Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.
